In [7]:
%pip install langchain-ollama pydantic langsmith python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
from pydantic import BaseModel, Field

class Response(BaseModel):
    question: str = Field(description="The question asked.")
    answer: str = Field(description="The answer to the question.")
    reasoning: str = Field(description="The reasoning behind the answer.")

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

system_prompt = """You are a helpful assistant. You will be given a question and you need to answer it step by step."""

template = """Question: {question}

Answer: Let's think step by step."""

parser = StrOutputParser()


prompt = ChatPromptTemplate.from_messages([
  ("system", system_prompt),
  ("human", template),
])

model = ChatOllama(model="deepseek-r1:7b", base_url="http://localhost:11434")
# .with_structured_output(Response)


In [14]:
from langchain_core.runnables import RunnableLambda

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: len(x.split()))

In [15]:
chain = prompt | model | parser | uppercase_output | count_words
response = chain.invoke({"question": "What is the capital of Thailand?"})
response

250

In [25]:
promtpt_template = ChatPromptTemplate.from_messages([
  ("system", "You are an expert product reviewer."),
  ("human", "List the main features of the product {product_name}."),
])

pros_template = ChatPromptTemplate.from_messages([
  ("system", "You are an expert product reviewer."),
  ("human", "Given these features: {features}, list the pros of these features."),
])


cons_template = ChatPromptTemplate.from_messages([
  ("system", "You are an expert product reviewer."),
  ("human", "Given these features: {features}, list the cons of these features."),
])

pros_chain = pros_template | model | parser
cons_chain = cons_template | model | parser

output_parser = RunnableLambda(lambda x: f"Pros: {x['pros']}, Cons: {x['cons']}")

product_chain = promtpt_template | model | parser | {
  "pros": pros_chain,
  "cons": cons_chain,
} | output_parser

product_chain.invoke({"product_name": "iPhone 14 Pro Max"})



"Pros: <think>\nAlright, so I need to evaluate the main features of the iPhone 14 Pro Max based on the given points. Let's break it down step by step.\n\nFirstly, storage options are important because having enough storage is crucial for storing all my photos, videos, and other files without running into issues. The iPhone 14 Pro offers up to 256GB, which seems decent, but the Pro Max goes even further with a 1TB option. That’s a significant jump and might be necessary if I have a lot of data.\n\nNext is the camera system. The Pro has three cameras: one main, one ultra-wide, and a depth sensor for Face ID. The Pro Max probably includes an additional wide-angle lens or other advanced features like a telephoto lens. More cameras mean better versatility in photography, which is great for capturing different styles and subjects.\n\nThe display quality being OLED makes sense because it offers sharp colors and low contrast ratios. A 6.1-inch display sounds decent, but I wonder if that's the 